In [1]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [2]:

api_wrapper=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper)

In [3]:
wiki.name


'wikipedia'

In [9]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
embedding_model=OllamaEmbeddings(model="gemma:2b")

C:\Users\user\AppData\Local\Temp\ipykernel_10380\2088588400.py:5: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embedding_model=OllamaEmbeddings(model="gemma:2b")


In [10]:
loader=WebBaseLoader("https://docs.smith.langchain.com/")
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vectordb=FAISS.from_documents(documents,embedding_model)
retriever=vectordb.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001EC535C20F0>, search_kwargs={})

In [11]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool=create_retriever_tool(retriever,"langsmith_search",
                      "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!")


In [12]:
retriever_tool.name


'langsmith_search'

In [13]:
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper=ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=200)
arxiv=ArxivQueryRun(api_wrapper=arxiv_wrapper)

In [14]:
arxiv.name

'arxiv'

In [15]:
tools=[wiki,arxiv,retriever_tool]
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'C:\\Users\\user\\AppData\\Roaming\\Python\\Python312\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=200)),
 Tool(name='langsmith_search', description='Search for information about LangSmith. For any questions about LangSmith, you must use this tool!', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x000001EC2E241800>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langch

In [16]:
from dotenv import load_dotenv

load_dotenv()
import os
os.environ['OPENAI_API_KEY']=os.getenv('OPENAI_API_KEY')
#from langchain_openai import ChatOpenAI
from langchain_community.llms import Ollama

#llm=ChatOpenAI(model="gpt-3.5-turbo-0125",temperature=0)

llm=Ollama(model="gemma:2b")


C:\Users\user\AppData\Local\Temp\ipykernel_10380\2417240471.py:11: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm=Ollama(model="gemma:2b")


In [24]:
from langchain.prompts import ChatPromptTemplate

# Create a proper prompt that includes 'agent_scratchpad'
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI assistant."),
    ("human", "{input}"),
    ("ai", "{agent_scratchpad}")  # This is the missing variable!
])


'''from langchain_core.prompts import ChatPromptTemplate
prompt=ChatPromptTemplate.from_messages(
    [
        ("system"," You are a University Chatbot. Please response to the queries"),
        ("user","Question:{question}")
    ]
)'''

'from langchain_core.prompts import ChatPromptTemplate\nprompt=ChatPromptTemplate.from_messages(\n    [\n        ("system"," You are a University Chatbot. Please response to the queries"),\n        ("user","Question:{question}")\n    ]\n)'

In [25]:
# AGENTS
from langchain.agents import create_openai_tools_agent
agent=create_openai_tools_agent(llm,tools,prompt)

In [26]:
## AGENT Executor
from langchain.agents import AgentExecutor

agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful AI assistant.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=['agent_scratchpad'], input_types={}, partial_variables={}, template='{agent_scratchpad}'), additional_kwargs={})])
| RunnableBinding(bound=Ollama(model='gemma:2b'), kwargs={'tools': [{'type': 'function', 'function': {'name': 'wikipedia', 'description': 'A wrapper around Wikipedia. Useful f

In [28]:
agent_executor.invoke({"input":"Tell me about LangSmith"})



> Entering new AgentExecutor chain...


ValueError: This output parser only works on ChatGeneration output

In [29]:
agent_executor.invoke({"input":"Tell me about Machine Learning"})



> Entering new AgentExecutor chain...


ValueError: This output parser only works on ChatGeneration output